[link dataset](https://www.kaggle.com/datasets/doctrinek/oxford-iiit-cats-extended-10k)

# Setup

### command line

In [1]:
# import the kaggle.json from kaggle API into colab
# do this command

# install kaggle library
!pip install kaggle
# make a directory named .kaggle
!mkdir ~/.kaggle
# copy the kaggle.json into this new directory
!cp kaggle.json ~/.kaggle/
# alocate the required permission for this file
!chmod 600 ~/.kaggle/kaggle.json
# download dataset
!kaggle datasets download doctrinek/oxford-iiit-cats-extended-10k

 98% 977M/993M [00:13<00:00, 117MB/s]
100% 993M/993M [00:13<00:00, 77.2MB/s]


In [2]:
!pip install split-folders
!pip install pillow

### import libraries

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.regularizers import l2
import numpy as np
import matplotlib.pyplot as plt
import pathlib, zipfile, os, splitfolders, datetime
from PIL import Image

### setup tensorboard

In [4]:
%load_ext tensorboard

In [5]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [6]:
!rm -rf ./logs/

In [7]:
tf.summary.create_file_writer("./logs/")

In [8]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

### extract zip file

In [9]:
zip_path = "/content/oxford-iiit-cats-extended-10k.zip"
zip_read = zipfile.ZipFile(zip_path, "r")
zip_read.extractall('/content/dataset')
zip_read.close()

In [10]:
os.listdir('/content/dataset/')

['CatBreedsRefined-v3']

### split into train and validation

In [11]:
original_dir = '/content/dataset/CatBreedsRefined-v3'

In [12]:
splitfolders.ratio(original_dir, output='/content/dataset/project', seed=6969, ratio=(0.8, 0.2))
train_dir = '/content/dataset/project/train'
validation_dir = '/content/dataset/project/val'

Copying files: 10257 files [00:07, 1453.51 files/s]


### explore data samples

In [13]:
def total_sample(directory):
  total = 0
  for folder in os.listdir(directory):
    folder_path = os.path.join(directory, folder)
    total += len(os.listdir(folder_path))

  return total

train_sample_length = total_sample(train_dir)
validation_sample_length = total_sample(validation_dir)
print(f"The train directory has {train_sample_length} samples")
print(f"The validation directory has {validation_sample_length} samples")
print(f"Which in total makes it {train_sample_length + validation_sample_length} samples")

The train directory has 8202 samples
The validation directory has 2055 samples
Which in total makes it 10257 samples


# Preprocess Data
```
rescale = 1.0/255,
shear_range=0.2,
zoom_range=0.2,
width_shift_range=0.2,
height_shift_range=0.2,
channel_shift_range=0.2,
horizontal_flip=True,
fill_mode='nearest'
```

In [29]:
batch_size = 16
target_size=(192,192)

train_datagen = ImageDataGenerator(
    rescale = 1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    channel_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    class_mode='categorical',
    target_size=target_size,
    batch_size=batch_size
)

validation_datagen = ImageDataGenerator(
    rescale=1.0/255
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    class_mode='categorical',
    target_size=target_size,
    batch_size=batch_size
)

Found 8202 images belonging to 12 classes.
Found 2055 images belonging to 12 classes.


# Model Creation

### callback function
- stops when acc & val_acc is >= 92%
- stops when acc / val_acc < max_acc for limit_acc epochs
- stops when loss / val_loss > 0.75 for limit_loss epochs


In [15]:
class SantaiDuluGakSih(tf.keras.callbacks.Callback):
  def __init__(self, sabar_acc=10, sabar_loss=10):
    super(SantaiDuluGakSih, self).__init__()
    self.sabar_acc = sabar_acc
    self.sabar_loss = sabar_loss
    self.limit_acc = sabar_acc
    self.limit_loss = sabar_loss
    self.max_acc = 0
    self.max_val_acc = 0

  def on_epoch_end(self, epoch, logs={}):
    self.max_acc = logs.get('accuracy') if logs.get('accuracy') > self.max_acc else self.max_acc

    self.max_val_acc = logs.get('val_accuracy') if logs.get('val_accuracy') > self.max_val_acc else self.max_val_acc

    if logs.get('accuracy')>=self.max_acc and logs.get('val_accuracy')>=self.max_val_acc:
      self.sabar_acc = self.limit_acc
    else:
      self.sabar_acc -= 1

    if logs.get('loss')>0.75 or logs.get('val_loss')>0.75:
      self.sabar_loss -= 1
    else:
      self.sabar_loss += 1

    if self.sabar_acc == 0:
      print(f"The model accuracy has been below {self.max_acc} and {self.max_val_acc} for {self.limit_acc} epochs, Stopping training immediatly!!!")
      self.model.stop_training = True
    elif self.sabar_loss == 0:
      print(f"The model loss has been above 75% for {self.limit_loss} epochs, Stopping training immediatly!!!")
      self.model.stop_training = True
    elif logs.get('accuracy') >= 0.92 and logs.get('val_accuracy') >= 0.92:
      print(f"The model accuracy has reached 92%, stopping training")
      self.model.stop_training = True

### transfer learning using MobileNetV2

In [16]:
"""model = tf.keras.Sequential()

# add mobile net as base layer
pre_trained_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_tensor=Input(shape=(224,224,3))
)

for layer in pre_trained_model.layers:
  layer.trainable = False

for layer in pre_trained_model.layers[-10:]:
  layer.trainable = True

model.add(pre_trained_model)

# add custom layers
model.add(Dropout(0.3))
model.add(Conv2D(72, (3,3), activation="relu"))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(512, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(Dense(64, activation="relu"))
model.add(Dense(12, activation="softmax"))

model.summary()"""

'model = tf.keras.Sequential()\n\n# add mobile net as base layer\npre_trained_model = MobileNetV2(\n    weights="imagenet",\n    include_top=False,\n    input_tensor=Input(shape=(224,224,3))\n)\n\nfor layer in pre_trained_model.layers:\n  layer.trainable = False\n\nfor layer in pre_trained_model.layers[-10:]:\n  layer.trainable = True\n\nmodel.add(pre_trained_model)\n\n# add custom layers\nmodel.add(Dropout(0.3))\nmodel.add(Conv2D(72, (3,3), activation="relu"))\nmodel.add(MaxPooling2D(2,2))\nmodel.add(Dropout(0.3))\nmodel.add(Flatten())\nmodel.add(Dense(512, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.001)))\nmodel.add(Dense(64, activation="relu"))\nmodel.add(Dense(12, activation="softmax"))\n\nmodel.summary()'

In [42]:
do = 0.3
model = tf.keras.models.Sequential([
    Conv2D(128, (3,3), activation="relu", input_shape=(192,192,3)),
    MaxPooling2D(2,2),
    Conv2D(32, (3,3), activation="relu"),
    MaxPooling2D(2,2),
    Dropout(do),
    Conv2D(128, (3,3), activation="relu"),
    MaxPooling2D(2,2),
    Conv2D(32, (3,3), activation="relu"),
    MaxPooling2D(2,2),
    Dropout(do),
    Flatten(),
    Dense(256, activation="relu"),
    Dense(64, activation="relu"),
    Dense(12, activation="softmax")
])

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_32 (Conv2D)          (None, 190, 190, 128)     3584      
                                                                 
 max_pooling2d_32 (MaxPooli  (None, 95, 95, 128)       0         
 ng2D)                                                           
                                                                 
 conv2d_33 (Conv2D)          (None, 93, 93, 32)        36896     
                                                                 
 max_pooling2d_33 (MaxPooli  (None, 46, 46, 32)        0         
 ng2D)                                                           
                                                                 
 dropout_12 (Dropout)        (None, 46, 46, 32)        0         
                                                                 
 conv2d_34 (Conv2D)          (None, 44, 44, 128)      

In [43]:
int_lr = 1e-2
#lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=int_lr, decay_steps=500, decay_rate=0.5)
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=int_lr),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
berhenti_bang = SantaiDuluGakSih(sabar_acc=4, sabar_loss=10)
modelku = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[berhenti_bang, tensorboard_callback],
    verbose=2
)

Epoch 1/20
513/513 - 162s - loss: 2.8060 - accuracy: 0.0813 - val_loss: 2.4858 - val_accuracy: 0.0847 - 162s/epoch - 317ms/step
Epoch 2/20
513/513 - 158s - loss: 2.4871 - accuracy: 0.0817 - val_loss: 2.4861 - val_accuracy: 0.0832 - 158s/epoch - 309ms/step
Epoch 3/20


# Model evaluation

### plot loss and accuracy

In [ ]:
# plot loss
plt.plot(modelku.history['loss'])
plt.plot(modelku.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

# plot acc
plt.plot(modelku.history['accuracy'])
plt.plot(modelku.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

### show tensorboard

In [ ]:
%tensorboard --logdir logs/fit

# Save model

In [ ]:
# menyimpan model dalam format saved model
export_dir = 'saved_model/'
tf.saved_model.save(model, export_dir)

# convert SavedModel menjadi vegs.tflite
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

tflite_model_file = pathlib.Path('vegs.tflite')
tflite_model_file.write_bytes(tflite_model)